In [1]:
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()
groq_key = os.getenv("GROQ")
tavily_key = os.getenv("TAVILY_API_KEY")

c:\Users\Public\anaconda3\envs\langchaingroq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
groq_model = ChatGroq(
    model="llama3-70b-8192",
    groq_api_key=groq_key)

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults(
    max_results=3,
    tavily_api_key=tavily_key,
    
)
search.invoke(
    "Where is the champions trophy 2025 played?"
)

[{'title': '2025 ICC Champions Trophy - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/2025_ICC_Champions_Trophy',
  'content': "The 2025 ICC Champions Trophy is the ninth edition of the ICC Champions Trophy. It is being hosted by Pakistan Cricket Board from 19 February to 9 March 2025. It features 15 matches being played across three venues in Pakistan and one in the United Arab Emirates. [...] On 19 December 2024, following an agreement between BCCI and PCB, the ICC in an update issued on India and Pakistan hosted matches at ICC events, established that the ICC Champions Trophy 2025 will be played across Pakistan and a neutral venue.[13][14] The ICC board confirmed that India and Pakistan matches hosted by either country at ICC events between 2024 and 2027 would be played at a neutral venue. This will also apply to the 2025 Women's Cricket World Cup (hosted by India; but only if [...] 2025 ICC Champions Trophy\nInternational cricket competitions in 2024–25\nInternational cricket

In [5]:
tools = [search]

### We will create an Agent now

In [6]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(
    groq_model,
    tools
)

In [8]:
from langchain_core.messages import HumanMessage
response = agent_executor.invoke(
    {
        "messages": [
            HumanMessage(
                content="Where is the champions trophy 2025 held?"
            )
        ]
    }
)
response['messages']

[HumanMessage(content='Where is the champions trophy 2025 held?', additional_kwargs={}, response_metadata={}, id='1143af5a-a86e-460c-a93c-e70e21ed658a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5096', 'function': {'arguments': '{"query":"ICC Champions Trophy 2025 venue"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 949, 'total_tokens': 1002, 'completion_time': 0.167790509, 'prompt_time': 0.031970558, 'queue_time': 0.060800800999999995, 'total_time': 0.199761067}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2e0feca3c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c5366aa6-b9d2-449c-b34f-5a2f1342d4f9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'ICC Champions Trophy 2025 venue'}, 'id': 'call_5096', 'type': 'tool_call'}], usage_metadata={'input_tokens': 949, 'output_tokens': 53, 'total_tokens': 1002}),
 ToolM

## Adding Memory

In [9]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [10]:
agent_executor_memory = create_react_agent(
    groq_model,
    tools,
    checkpointer=memory
)

In [12]:
config = {
    "configurable": {
        "thread_id": "001"
    }
}
## Multiple user can use the application, So we have id for each user.

In [15]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Who won the champions trophy 2025?"
            )
        ]
    }, 
    config
):
    print(chunk)
    print("*************************************************************")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jea4', 'function': {'arguments': '{"query":"Champions Trophy 2025 winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 948, 'total_tokens': 1001, 'completion_time': 0.151428571, 'prompt_time': 0.03062285, 'queue_time': 0.067230031, 'total_time': 0.182051421}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2e0feca3c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d53e6910-559d-475f-9da6-5840e4ae0647-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Champions Trophy 2025 winner'}, 'id': 'call_jea4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 948, 'output_tokens': 53, 'total_tokens': 1001})]}}
*************************************************************
{'tools': {'messages': [ToolMessage(content='[{"title": "India beat New Zealand by four w

In [18]:
for chunk in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="Who were the top stars of that winner team of champions trophy 2025?"
            )
        ]
    }, 
    config
):
    print(chunk)
    print("*************************************************************")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rfdx', 'function': {'arguments': '{"query":"Champions Trophy 2025 winner team"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 955, 'total_tokens': 1009, 'completion_time': 0.179925827, 'prompt_time': 0.03076065, 'queue_time': 0.058305569, 'total_time': 0.210686477}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2e0feca3c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8b9ec1b5-1bf2-4522-9e67-6937ba07da3c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Champions Trophy 2025 winner team'}, 'id': 'call_rfdx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 955, 'output_tokens': 54, 'total_tokens': 1009})]}}
*************************************************************
{'tools': {'messages': [ToolMessage(content='[{"title": "ICC announces Champion